#  3D Mohr Plots and Slip Tendancy Analysis

## 3D Mohr plots

The ratio of shear to normal stress resolved onto a fracture can be represented on 3D Mohr circles.

A figure with parts of the Mohr plot labelled - sigma1, sigma2 and sigma3


In [ ]:
# Mohr plot code example



## Slip Tendancy Analysis

Through a series of algebraic stress tensor rotations, the effective stress tensor can be used to determine the shear ($\tau$) and normal stress ($\sigma_n$) resolved onto a plane. 

Barton et al. (1995) and others have demonstrated that when the ratio of shear to normal stress is high, then there is a greater likelihood that a fracture has recently re-sheared and is therefore open to fluid flow. We established in 1-openhole-logs.ipynb that there are thermally enhanced hoop stresses at the wall of a geothermal borehole. These conditions mean that many more discontinuities are open at the borehole wall than in the reservoir. Calculating the ratio of shear to noraml stress on each fracture enables us to identify the population that is likely open to fluid flow in the reservoir. 

Overview of workflow:
- Define the input fracture geometry (strike/dip), geomechanical model, and pore pressure, noting that the orientation of the stress field is described in terms of Euler angles
- Convert the stress tensor to effective stress
- Transform the stress tensor to geographic coordinates using the Euler angles
- Transform the stress tensor into the coordinate system of the fault/fracture plane, and then select the normal stress
- Transform the stress tensor so it is aligned with the fracture rake, and then select the absolute value of shear stress

The rake is defined here as a vector down the fracture plane perpendicular from the strike, such that we assume pure dip-slip. When characterising a fault in the field, a rake is measured from striations on the fault surface that indicate slip sense and may not be purely dip-slip.   
 
The method I use to calculate shear and normal stress is the tensor transformation process outlined by Zoback (2010, pp 156-157).  

## Case study

The slip tendancy workflow described above was run for NM-B is a geothermal well in the Taupō Volcanic Zone, New Zealand. The context and methods are described in detail in Wallis et al. (2020 - submitted) and summarised here.



<img src="images/mohr-plots-02.png" alt="Drawing" style="width: 300px;"/>


<img src="images/mohr-plots-01.png" alt="Drawing" style="width: 1000px;"/>


<img src="images/mohr-plots-03.png" alt="Drawing" style="width: 700px;"/>

When plotted by depth and combined with geologic context, the slip tendency modelling reveals that the upper two feedzones are spatially correlated with clusters of fractures that have a high slip tendency. When combined with the apparent stress field rotations, it is likely that these clusters are associated with recent fault rupture. 

A third feedzone is correlated with a volcaniclastic breccia that was revealed by textural analysis of the borehole image log. Cuttings analysis only indicated that this unit was comprised of andesite lava, likely because cuttings are mm scale and the andesite lava clasts are bolder size. Apparent stress tensor rotation around this depth may be associated with either recent faulting or stress anisotropy related to the sedimentary layers. 

The deepest feedzone is extremely small. Because of the pressure difference between a well filled with cool fluid and the hot reservoir, completion testing empathises deep feedzones. It is not unusual for low-magnitude, deep feedzones to contribute minimally or not at all if a well is flowed. For more information on geothermal well test methodologies, refer to Zarrouk and McLean (2019). Analysis of the geologic context of this deep feedzone has yet to reveal a clear mechanism controlling its location.

<img src="images/slip-tendancy-01.png" alt="Drawing" style="width: 900px;"/>

### Other reccomended case studies:

- The coso paper
- Barton's seminarl work

## Citations

Wallis, I.C., Rowland, J. V. and Dempsey, D. E., Allan, G., Sidik, R., Martikno, R., McLean, K., Sihotang, M., Azis, M. and Baroek, M. 2020 (submitted) Approaches to imaging feedzone diversity with case studies from Sumatra, Indonesia, and the Taupō Volcanic Zone, New Zealand. New Zealand Geothermal Workshop: Waitangi, New Zealand.

Zarrouk, S. J., and McLean, K., 2019, Geothermal well test analysis : fundamentals, applications and advanced techniques, London : Academic Press. 2019.

Zoback, M. D., 2010, Reservoir Geomechanics, Cambridge University Press.



In [ ]:
%load_ext watermark
print('author:')
%watermark -a "Irene Wallis" -d
print('\n')
print('requirements:')
%watermark -v -p pandas,matplotlib,mplstereonet,seaborn,watermark,fractoolbox 
print('\n')
%watermark -g